In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
from functools import reduce

In [3]:
demoDF = pd.read_csv('../input/test-data/DEMO19Q1.txt', sep=',', error_bad_lines=False, index_col=False, low_memory=False)
drugDF = pd.read_csv('../input/test-data/DRUG19Q1.txt', sep=',', error_bad_lines=False, index_col=False, low_memory=False)
reacDF = pd.read_csv('../input/test-data/REAC19Q1.txt', sep=',', error_bad_lines=False, index_col=False, low_memory=False)
outcDF = pd.read_csv('../input/test-data/OUTC19Q1.txt', sep=',', error_bad_lines=False, index_col=False, low_memory=False)
indiDF = pd.read_csv('../input/test-data/INDI19Q1.txt', sep=',', error_bad_lines=False, index_col=False, low_memory=False)


FileNotFoundError: [Errno 2] File ../input/test-data/DEMO19Q1.txt does not exist: '../input/test-data/DEMO19Q1.txt'

In [5]:
indiDF.head(10)

,# primaryid,caseid,indi_drug_seq,indi_pt
0,1,3894168,1,Parkinsonism
1,2,3894168,7,Parkinsonism
2,3,6132163,2,Non-Hodgkin's lymphoma
3,4,6445504,7,Nausea
4,5,6671018,8,Foetal exposure during pregnancy


In [6]:
# drugDF = drugDF.groupby(['# primaryid', 'caseid', 'drugname', 'dose_form', 'route'])['dose_vbm'].apply(','.join).reset_index()
# reacDF = reacDF.groupby(['# primaryid', 'caseid'])['pt'].apply(','.join).reset_index()
# outcDF = outcDF.groupby(['# primaryid', 'caseid'])['outc_cod'].apply(','.join).reset_index()
# indiDF = indiDF.groupby(['# primaryid', 'caseid'])['indi_pt'].apply(set).apply(','.join).reset_index()

In [7]:
reacDF.head(10)

,# primaryid,caseid,pt,drug_rec_act
0,1,3894168,Agitation,NaN
1,2,9504195,Bile duct obstruction,NaN
2,3,9504195,Cholangitis acute,NaN
3,3,9504195,Decreased appetite,NaN
4,3,9504195,Depression,NaN
5,6,9504195,Duodenal ulcer,NaN
6,7,9504195,Dyspnoea,NaN
7,8,9504195,Road traffic accident,NaN


In [8]:
outcDF.head()

,# primaryid,caseid,outc_cod
0,1,3762458,HO
1,2,3764613,HO
2,2,5959328,OT
3,2,5996530,LT
4,5,6110401,DE


In [9]:
demoDF.head()

,# primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,mfr_num,mfr_sndr,age,age_cod,age_grp,sex,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
0,1,4022386,3,F,0,20191016,20031000,GB-JNJFOC-20031000825,JOHNSON AND JOHNSON,52,YR,NaN,M,0,NaN,20191000,NaN,OT,GB,GB
1,2,4076188,6,F,200303,20051109,20040100,PHBS2003JP07309,NOVARTIS,80,YR,NaN,M,0,NaN,20190200,NaN,OT,JP,JP
2,3,4122546,4,F,0,20040428,20040400,PHBS2004DE04597,NOVARTIS,79,YR,NaN,F,0,NaN,20190400,NaN,CN,FR,DE
3,4,4208085,4,F,20020600,20041020,20040900,PHBS2004JP11897,NOVARTIS,86,YR,NaN,F,0,NaN,20190100,NaN,MD,JP,JP
4,5,8299276,6,F,20111100,20150312,20111200,DE-ROCHE-1022692,ROCHE,62,YR,NaN,F,84,KG,20191200,NaN,MD,DE,DE


Demographic Filtering

In [10]:
merged_left = pd.merge(left=demoDF, right=reacDF, how='left', left_on='# primaryid', right_on='# primaryid')
merged_left.drop("caseid_y",axis=1,inplace=True)
merged_left

,# primaryid,caseid_x,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,mfr_num,mfr_sndr,age,...,sex,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country,pt,drug_rec_act
0,1,4022386,3,F,0,20191016,20031000,GB-JNJFOC-20031000825,JOHNSON AND JOHNSON,52,...,M,0,NaN,20191000,NaN,OT,GB,GB,Agitation,NaN
1,2,4076188,6,F,200303,20051109,20040100,PHBS2003JP07309,NOVARTIS,80,...,M,0,NaN,20190200,NaN,OT,JP,JP,Bile duct obstruction,NaN
2,3,4122546,4,F,0,20040428,20040400,PHBS2004DE04597,NOVARTIS,79,...,F,0,NaN,20190400,NaN,CN,FR,DE,Cholangitis acute,NaN
3,3,4122546,4,F,0,20040428,20040400,PHBS2004DE04597,NOVARTIS,79,...,F,0,NaN,20190400,NaN,CN,FR,DE,Decreased appetite,NaN
4,3,4122546,4,F,0,20040428,20040400,PHBS2004DE04597,NOVARTIS,79,...,F,0,NaN,20190400,NaN,CN,FR,DE,Depression,NaN
5,4,4208085,4,F,20020600,20041020,20040900,PHBS2004JP11897,NOVARTIS,86,...,F,0,NaN,20190100,NaN,MD,JP,JP,NaN,NaN
6,5,8299276,6,F,20111100,20150312,20111200,DE-ROCHE-1022692,ROCHE,62,...,F,84,KG,20191200,NaN,MD,DE,DE,NaN,NaN
7,6,8828837,4,F,201203,20190531,20121000,CA-ROCHE-CID000000002173273,ROCHE,54,...,M,89,KG,20190600,NaN,CN,CA,CA,Duodenal ulcer,NaN
8,7,11806747,4,F,20060500,20191227,20151200,US-PFIZER INC-2015421690,PFIZER,49,...,M,0,NaN,20191200,NaN,LW,US,US,Dyspnoea,NaN


In [11]:
merged_left.columns

Index(['# primaryid', 'caseid_x', 'caseversion', 'i_f_code', 'event_dt',
       'mfr_dt', 'init_fda_dt', 'mfr_num', 'mfr_sndr', 'age', 'age_cod',
       'age_grp', 'sex', 'wt', 'wt_cod', 'rept_dt', 'to_mfr', 'occp_cod',
       'reporter_country', 'occr_country', 'pt', 'drug_rec_act'],
      dtype='object')

In [12]:
demographic_filtering= merged_left[['# primaryid','age','occr_country','pt']][merged_left['occr_country']=='JP']
demographic_filtering

,# primaryid,age,occr_country,pt
1,2,80,JP,Bile duct obstruction
5,4,86,JP,NaN


Demographic Filtering/ Case Id

In [13]:
merged_left1 = pd.merge(left=demoDF, right=reacDF, how='outer', left_on='caseid', right_on='caseid')
merged_left1.drop("# primaryid_y",axis=1,inplace=True)
merged_left1

,# primaryid_x,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,mfr_num,mfr_sndr,age,...,sex,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country,pt,drug_rec_act
0,1.0,4022386,3.0,F,0.0,20191016.0,20031000.0,GB-JNJFOC-20031000825,JOHNSON AND JOHNSON,52.0,...,M,0.0,NaN,20191000.0,NaN,OT,GB,GB,NaN,NaN
1,2.0,4076188,6.0,F,200303.0,20051109.0,20040100.0,PHBS2003JP07309,NOVARTIS,80.0,...,M,0.0,NaN,20190200.0,NaN,OT,JP,JP,NaN,NaN
2,3.0,4122546,4.0,F,0.0,20040428.0,20040400.0,PHBS2004DE04597,NOVARTIS,79.0,...,F,0.0,NaN,20190400.0,NaN,CN,FR,DE,NaN,NaN
3,4.0,4208085,4.0,F,20020600.0,20041020.0,20040900.0,PHBS2004JP11897,NOVARTIS,86.0,...,F,0.0,NaN,20190100.0,NaN,MD,JP,JP,NaN,NaN
4,5.0,8299276,6.0,F,20111100.0,20150312.0,20111200.0,DE-ROCHE-1022692,ROCHE,62.0,...,F,84.0,KG,20191200.0,NaN,MD,DE,DE,NaN,NaN
5,6.0,8828837,4.0,F,201203.0,20190531.0,20121000.0,CA-ROCHE-CID000000002173273,ROCHE,54.0,...,M,89.0,KG,20190600.0,NaN,CN,CA,CA,NaN,NaN
6,7.0,11806747,4.0,F,20060500.0,20191227.0,20151200.0,US-PFIZER INC-2015421690,PFIZER,49.0,...,M,0.0,NaN,20191200.0,NaN,LW,US,US,NaN,NaN
7,NaN,3894168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Agitation,NaN
8,NaN,9504195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bile duct obstruction,NaN
9,NaN,9504195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cholangitis acute,NaN


In [14]:
demographic_filtering= merged_left1[['caseid','age','occr_country','pt']][merged_left1['occr_country']=='JP']
demographic_filtering

,caseid,age,occr_country,pt
1,4076188,80.0,JP,NaN
3,4208085,86.0,JP,NaN


Number of Reports/ Case Id

In [15]:
drugDF.head(10)

,# primaryid,caseid,drug_seq,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,exp_dt,dose_amt,dose_unit,dose_form,dose_freq
0,1,3894168,1,BROMOCRIPTINE MESYLATE,BROMOCRIPTINE MESYLATE,1,Oral,"10 MG, TID",0,NaN,Y,NaN,NaN,10.0,MG,NaN,TID
1,1,3894168,2,BROMOCRIPTINE MESYLATE,BROMOCRIPTINE MESYLATE,1,Oral,"7.5 MG, TID",0,NaN,Y,NaN,NaN,7.5,MG,NaN,TID
2,1,3894168,7,LEVODOPA+BENSERAZIDE,BENSERAZIDE\LEVODOPA,1,Oral,"125 MG, 6QD",0,NaN,NaN,NaN,NaN,125.0,MG,NaN,NaN
3,4,4022386,1,"INFLIXIMAB, RECOMBINANT",INFLIXIMAB,1,Intravenous (not otherwise specified),"WEEKS 0, 2, 6, 10, 18, 26, AND 34 CYCLICAL",0,NaN,U,U,NaN,10.0,MG/KG,SOLUTION FOR INFUSION,NaN
4,5,3894168,13,SELEGILINE,SELEGILINE,1,Unknown,"15 MG, UNK",0,NaN,U,NaN,NaN,15.0,MG,NaN,NaN
5,6,11259551,3,DICLOFENAC SODIUM.,DICLOFENAC SODIUM,1,NaN,"100 MG, ONCE DAILY (QD)",0,NaN,NaN,U,NaN,100.0,MG,NaN,QD
6,7,11468194,1,LIPITOR,ATORVASTATIN CALCIUM,1,Oral,"20 MG, DAILY",0,NaN,U,NaN,NaN,20.0,MG,FILM-COATED TABLET,NaN


In [16]:
n_by_state = drugDF.groupby(["drugname"])["drugname"].count()
n_by_state

drugname
BROMOCRIPTINE MESYLATE     2
DICLOFENAC SODIUM.         1
INFLIXIMAB, RECOMBINANT    1
LEVODOPA+BENSERAZIDE       1
LIPITOR                    1
SELEGILINE                 1
Name: drugname, dtype: int64

In [17]:
testingdf = pd.DataFrame(n_by_state)
testingdf

,drugname
drugname,
BROMOCRIPTINE MESYLATE,2
DICLOFENAC SODIUM.,1
"INFLIXIMAB, RECOMBINANT",1
LEVODOPA+BENSERAZIDE,1
LIPITOR,1
SELEGILINE,1


In [18]:
testingdf= testingdf.add_suffix('_Count').reset_index()


In [19]:
print(testingdf)

                  drugname  drugname_Count
0   BROMOCRIPTINE MESYLATE               2
1       DICLOFENAC SODIUM.               1
2  INFLIXIMAB, RECOMBINANT               1
3     LEVODOPA+BENSERAZIDE               1
4                  LIPITOR               1
5               SELEGILINE               1


In [20]:
drug_filtering= testingdf[['drugname','drugname_Count']][testingdf['drugname']=='BROMOCRIPTINE MESYLATE']
drug_filtering

,drugname,drugname_Count
0,BROMOCRIPTINE MESYLATE,2


In [21]:
drugDF1 = pd.read_csv('../input/fda1year/Drug19.csv', sep=';', error_bad_lines=False, index_col=False, low_memory=False)

In [22]:
drugDF1.head()

,primaryid,caseid,drug_seq,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,exp_dt,dose_amt,dose_unit,dose_form,dose_freq
0,38941682,3894168,1,BROMOCRIPTINE MESYLATE,BROMOCRIPTINE MESYLATE,1,Oral,"10 MG, TID",0.0,NaN,Y,NaN,NaN,10.0,MG,NaN,TID
1,38941682,3894168,2,BROMOCRIPTINE MESYLATE,BROMOCRIPTINE MESYLATE,1,Oral,"7.5 MG, TID",0.0,NaN,Y,NaN,NaN,7.5,MG,NaN,TID
2,38941682,3894168,3,BROMOCRIPTINE MESYLATE,BROMOCRIPTINE MESYLATE,1,Oral,10-7.5-10 MG/DAY,0.0,NaN,Y,NaN,NaN,0.0,NaN,NaN,NaN
3,38941682,3894168,4,BROMOCRIPTINE MESYLATE,BROMOCRIPTINE MESYLATE,1,Unknown,"22.5 MG, QD",0.0,NaN,Y,NaN,NaN,22.5,MG,NaN,QD
4,38941682,3894168,5,BROMOCRIPTINE MESYLATE,BROMOCRIPTINE MESYLATE,1,Unknown,"30 MG, UNK",0.0,NaN,Y,NaN,NaN,30.0,MG,NaN,NaN


In [23]:
n_by_state1 = drugDF1.groupby(["drugname"])["drugname"].count()
n_by_state1

drugname
 AMBIEN 10MG AT BEDTIME               1
 AMPHETAMINE AND DEXTROAMPHETAMINE    1
 ATORVASTATIN                         1
 CALCIUMD                             1
 CARVEDIOL                            1
                                     ..
zyrtec 10mg otc                       1
zyrtec allergy tab 10 mg              1
zyrtek                                1
zytiga tab 5090 mg                    1
zytrec allergy 10mg tablets           1
Name: drugname, Length: 126898, dtype: int64

In [24]:
demoDF1 = pd.read_csv('../input/fda1year/Demo19.csv', sep=';', error_bad_lines=False, index_col=False, low_memory=False)

In [25]:
demoDF1

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,mfr_num,mfr_sndr,age,age_cod,age_grp,sex,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
0,37624583,3762458,3,F,20020100.0,20020408.0,20020200.0,PHBS2002JP01696,NOVARTIS,36.0,YR,NaN,F,44.0,KG,20190200.0,NaN,MD,JP,JP
1,37646134,3764613,4,F,20020100.0,20020405.0,20020200.0,PHBS2002JP01755,NOVARTIS,70.0,YR,NaN,F,66.0,KG,20190100.0,NaN,MD,JP,JP
2,37969093,3796909,3,F,20020100.0,20020605.0,20020500.0,PHBS2002JP03115,NOVARTIS,68.0,YR,NaN,F,45.5,KG,20190200.0,NaN,MD,JP,JP
3,38394683,3839468,3,F,20020700.0,20030204.0,20020900.0,PHBS2002JP09941,NOVARTIS,30.0,YR,NaN,M,58.5,KG,20190100.0,NaN,MD,JP,JP
4,38941682,3894168,2,F,0.0,20190809.0,20030100.0,PHBS2003DE00603,SANDOZ,50.0,YR,NaN,M,0.0,NaN,20190800.0,NaN,OT,DE,DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727291,1691418417,16914184,17,F,20191000.0,20191225.0,20191000.0,JP-EISAI MEDICAL RESEARCH-EC-2019-063594,EISAI,51.0,YR,NaN,F,52.7,KG,20191200.0,NaN,MD,JP,JP
1727292,1698091610,16980916,10,F,20191000.0,20191120.0,20191000.0,FR-EISAI MEDICAL RESEARCH-EC-2019-064437,EISAI,88.0,YR,NaN,F,64.0,KG,20191100.0,NaN,MD,FR,FR
1727293,1700149411,17001494,11,F,20191000.0,20191223.0,20191100.0,JP-EISAI MEDICAL RESEARCH-EC-2019-064738,EISAI,70.0,YR,NaN,M,63.8,KG,20191200.0,NaN,MD,JP,JP
1727294,9474120130,9474120,130,F,201305.0,20190905.0,20130800.0,PHHY2013CA043405,NOVARTIS,56.0,YR,NaN,M,0.0,NaN,20190900.0,NaN,MD,CA,CA


In [26]:
demoDF1_ = demoDF1.set_index(["primaryid"], drop=True)

In [27]:
demoDF1_

,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,mfr_num,mfr_sndr,age,age_cod,age_grp,sex,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
primaryid,,,,,,,,,,,,,,,,,,,
37624583,3762458,3,F,20020100.0,20020408.0,20020200.0,PHBS2002JP01696,NOVARTIS,36.0,YR,NaN,F,44.0,KG,20190200.0,NaN,MD,JP,JP
37646134,3764613,4,F,20020100.0,20020405.0,20020200.0,PHBS2002JP01755,NOVARTIS,70.0,YR,NaN,F,66.0,KG,20190100.0,NaN,MD,JP,JP
37969093,3796909,3,F,20020100.0,20020605.0,20020500.0,PHBS2002JP03115,NOVARTIS,68.0,YR,NaN,F,45.5,KG,20190200.0,NaN,MD,JP,JP
38394683,3839468,3,F,20020700.0,20030204.0,20020900.0,PHBS2002JP09941,NOVARTIS,30.0,YR,NaN,M,58.5,KG,20190100.0,NaN,MD,JP,JP
38941682,3894168,2,F,0.0,20190809.0,20030100.0,PHBS2003DE00603,SANDOZ,50.0,YR,NaN,M,0.0,NaN,20190800.0,NaN,OT,DE,DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691418417,16914184,17,F,20191000.0,20191225.0,20191000.0,JP-EISAI MEDICAL RESEARCH-EC-2019-063594,EISAI,51.0,YR,NaN,F,52.7,KG,20191200.0,NaN,MD,JP,JP
1698091610,16980916,10,F,20191000.0,20191120.0,20191000.0,FR-EISAI MEDICAL RESEARCH-EC-2019-064437,EISAI,88.0,YR,NaN,F,64.0,KG,20191100.0,NaN,MD,FR,FR
1700149411,17001494,11,F,20191000.0,20191223.0,20191100.0,JP-EISAI MEDICAL RESEARCH-EC-2019-064738,EISAI,70.0,YR,NaN,M,63.8,KG,20191200.0,NaN,MD,JP,JP


In [28]:
testingdf1 = pd.DataFrame(n_by_state1)

In [29]:
testingdf1= testingdf1.add_suffix('_Count').reset_index()
testingdf1.sort_values(by=['drugname'],inplace=True)

In [30]:
testingdf1

,drugname,drugname_Count
0,AMBIEN 10MG AT BEDTIME,1
1,AMPHETAMINE AND DEXTROAMPHETAMINE,1
2,ATORVASTATIN,1
3,CALCIUMD,1
4,CARVEDIOL,1
...,...,...
126893,zyrtec 10mg otc,1
126894,zyrtec allergy tab 10 mg,1
126895,zyrtek,1
126896,zytiga tab 5090 mg,1


In [31]:
drug_filtering1= testingdf1[['drugname','drugname_Count']][testingdf1['drugname']=='BROMOCRIPTINE MESYLATE']
drug_filtering1

,drugname,drugname_Count
16404,BROMOCRIPTINE MESYLATE,16


In [32]:
testingdf1["Intensity"] = 'Low'
testingdf1.loc[testingdf1["drugname_Count"]>3, "Intensity"]= "Mid"
testingdf1.loc[testingdf1["drugname_Count"]>5, "Intensity"]= "High"

In [33]:
testingdf1

,drugname,drugname_Count,Intensity
0,AMBIEN 10MG AT BEDTIME,1,Low
1,AMPHETAMINE AND DEXTROAMPHETAMINE,1,Low
2,ATORVASTATIN,1,Low
3,CALCIUMD,1,Low
4,CARVEDIOL,1,Low
...,...,...,...
126893,zyrtec 10mg otc,1,Low
126894,zyrtec allergy tab 10 mg,1,Low
126895,zyrtek,1,Low
126896,zytiga tab 5090 mg,1,Low


In [34]:
drug_filtering2= testingdf1[['drugname','Intensity']][testingdf1['Intensity']=='High']
drug_filtering2

,drugname,Intensity
33,+PREDNISONE TAB 20MG,High
52,0.9% SODIUM CHLORIDE INJECTION,High
53,0.9% SODIUM CHLORIDE INJECTION USP (250 ML),High
58,0.9% Sodium Chloride Injection,High
59,0.9% Sodium Chloride Injection USP (100 mL),High
...,...,...
126829,zarzio,High
126831,zelitrex,High
126862,zoledronic acid,High
126882,zopiclon,High


----------------------------------------------------------------**-------------------------------**

In [35]:
# merged_drug=  pd.merge(left=demoDF, right=drugDF, how='left', left_on='# primaryid', right_on='# primaryid')
# merged_drug.drop("caseid_y",axis=1,inplace=True)
# merged_drug

In [ ]:
print('Merging all 4 dataframes')
# Raw dataframe with duplicate rows (Most hangtime occurs here)
dfs = [demoDF, drugDF, reacDF, outcDF, indiDF]
mergedDF =reduce(lambda left,right: pd.merge(left,right,on=['# primaryid','caseid']), dfs)


In [6]:
mergedDF.columns

Index(['caseversion', 'i_f_code', 'event_dt', 'mfr_dt', 'init_fda_dt',
       'mfr_num', 'mfr_sndr', 'age', 'age_cod', 'age_grp', 'sex', 'wt',
       'wt_cod', 'rept_dt', 'to_mfr', 'occp_cod', 'reporter_country',
       'occr_country', 'drugname', 'dose_form', 'route', 'dose_vbm', 'pt',
       'outc_cod', '# primaryid', 'caseid', 'indi_pt'],
      dtype='object')

In [8]:
# Drop any duplicate rows with same values across all columns
mergedDF = mergedDF.iloc[mergedDF.astype(str).drop_duplicates().index]

# Sort rows by DRUGNAME alphanumerically
mergedDF.sort_values(by='drugname', inplace=True)

# Grabbing significant fields to group
mergedDF.fillna('null', inplace=True)

In [5]:
# drugDF['dose_vbm'] = drugDF['dose_vbm'].astype(str)
# drugDF = drugDF.groupby(['primaryid', 'caseid', 'drugname', 'dose_form', 'route'])['dose_vbm']
# reacDF = reacDF.groupby(['primaryid', 'caseid'])['pt']
# outcDF = outcDF.groupby(['primaryid', 'caseid'])['outc_cod']
# indiDF = indiDF.groupby(['primaryid', 'caseid'])['indi_pt']


In [ ]:
# mergedDF = mergedDF.iloc[mergedDF.astype(str).drop_duplicates().index]

# Sort rows by DRUGNAME alphanumerically
mergedDF.sort_values(by='drugname', inplace=True)

# Grabbing significant fields to group
mergedDF.fillna('null', inplace=True)


In [1]:
import glob
import os
import pandas as pd
from functools import reduce
import json
from collections import OrderedDict
import operator

In [2]:
demoDF1_ = pd.read_csv('Demo19.csv', sep=';', error_bad_lines=False, index_col=False, low_memory=False)
drugDF1 = pd.read_csv('Drug19.csv', sep=';', error_bad_lines=False, index_col=False, low_memory=False)

In [3]:
demoDF1_ = demoDF1_.set_index("primaryid",drop=True)

In [4]:
demoDF1_

,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,mfr_num,mfr_sndr,age,age_cod,age_grp,sex,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
primaryid,,,,,,,,,,,,,,,,,,,
37624583,3762458,3,F,20020100.0,20020408.0,20020200.0,PHBS2002JP01696,NOVARTIS,36.0,YR,NaN,F,44.0,KG,20190200.0,NaN,MD,JP,JP
37646134,3764613,4,F,20020100.0,20020405.0,20020200.0,PHBS2002JP01755,NOVARTIS,70.0,YR,NaN,F,66.0,KG,20190100.0,NaN,MD,JP,JP
37969093,3796909,3,F,20020100.0,20020605.0,20020500.0,PHBS2002JP03115,NOVARTIS,68.0,YR,NaN,F,45.5,KG,20190200.0,NaN,MD,JP,JP
38394683,3839468,3,F,20020700.0,20030204.0,20020900.0,PHBS2002JP09941,NOVARTIS,30.0,YR,NaN,M,58.5,KG,20190100.0,NaN,MD,JP,JP
38941682,3894168,2,F,0.0,20190809.0,20030100.0,PHBS2003DE00603,SANDOZ,50.0,YR,NaN,M,0.0,NaN,20190800.0,NaN,OT,DE,DE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1691418417,16914184,17,F,20191000.0,20191225.0,20191000.0,JP-EISAI MEDICAL RESEARCH-EC-2019-063594,EISAI,51.0,YR,NaN,F,52.7,KG,20191200.0,NaN,MD,JP,JP
1698091610,16980916,10,F,20191000.0,20191120.0,20191000.0,FR-EISAI MEDICAL RESEARCH-EC-2019-064437,EISAI,88.0,YR,NaN,F,64.0,KG,20191100.0,NaN,MD,FR,FR
1700149411,17001494,11,F,20191000.0,20191223.0,20191100.0,JP-EISAI MEDICAL RESEARCH-EC-2019-064738,EISAI,70.0,YR,NaN,M,63.8,KG,20191200.0,NaN,MD,JP,JP


In [5]:
k = pd.DataFrame(drugDF1['drugname'].value_counts(ascending=True))
k

,drugname
sodium chloride 3% inhalation solution,1
Pred-Clysma,1
SULFA/TRIMETH 800/160 MG,1
FENTANYL CITRATE/BUPIVACAINE HYDROCHLORIDE,1
Insulin glargine 40 units daily,1
...,...
LYRICA,44719
REVLIMID,54318
PREDNISONE.,54415
NEXIUM,55591


In [6]:
k[k['drugname'] == 5]

,drugname
Citalopram Oral Drops,5
EDICIN,5
ATROPINE SULFATE 1.0% Sol (0015B),5
RESTAMIN #1,5
Elocom,5
...,...
TEVA TELMISARTAN,5
HEPARIN LEO,5
HYDROXOCOBALAMIN ACETATE.,5
ACETOMINOPHEN,5


In [7]:
drugDF1[drugDF1['drugname'] == 'SALAZOPIRINA']

,primaryid,caseid,drug_seq,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,exp_dt,dose_amt,dose_unit,dose_form,dose_freq
36419,111223355,11122335,7,SALAZOPIRINA,SULFASALAZINE,1,Oral,NaN,0.0,NaN,NaN,NaN,NaN,500.0,MG,NaN,NaN
457489,144785407,14478540,17,SALAZOPIRINA,SULFASALAZINE,1,NaN,UNK,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1865307,159079631,15907963,7,SALAZOPIRINA,SULFASALAZINE,1,Oral,NaN,0.0,NaN,NaN,NaN,NaN,500.0,MG,NaN,NaN
1966930,159350762,15935076,3,SALAZOPIRINA,SULFASALAZINE,1,Oral,NaN,0.0,NaN,NaN,NaN,NaN,1.0,DF,NaN,Q12H
2453449,161660871,16166087,2,SALAZOPIRINA,SULFASALAZINE,1,Oral,IN THE MORNING AND AT NIGHT,0.0,NaN,NaN,NaN,NaN,0.0,NaN,TABLET,NaN


In [45]:
import requests
from statistics import mean 


weightDict = {
    'KG': 1,
    'LBS': 2.2,
    'GMS': 1000,
  }


ageDict = {
    'DEC': 0.1,
    'YR': 1,
    'MON': 12,
    'WK': 52,
    'DY': 365,
    'HR': 8760
  }

# Main function of matching drugname with demographics, count no of reports, and returing the rxCUI code

def get_drug_reports(drugname):
    print(drugname)
    # Iterating over every row in drugtable and match it with demographic table
    merged_df = demoDF1_[demoDF1_.index.isin(drugDF1[drugDF1['drugname'] == drugname]['primaryid'].values)]
    print('shape: ',merged_df.shape)
    print(merged_df.head())
    for index,row in merged_df.iterrows():
        val = row['age']/ageDict[row['age_cod']]
        merged_df.loc[index,'age'] = val
    merged_df=merged_df.where(pd.notnull(merged_df), None)
    merged_df_= merged_df.to_dict()


# Getting the rxCUI code

    rawResponse = requests.get('https://rxnav.nlm.nih.gov/REST/rxcui.json?name=' + drugname)
    res = rawResponse.json()
    if not 'rxnormId' in res['idGroup']:
        merged_df_["id"] = "Not Found"
    else: 
        rxCUICode = res['idGroup']['rxnormId'][0]
        merged_df_["id"] = rxCUICode
                
    merged_df_["drug name"] = drugname
    
# Counting length of the returned df to get the number of reports   
    merged_df_["No of Reports"] = len(merged_df_['caseid'])
    
### Changing the values
    ## i_f_code

    dict_i_f_code = get_dict(merged_df_['i_f_code'])
#     print(dict_i_f_code)
    merged_df_['i_f_code'] = dict_i_f_code
    
    ## age
    age_dict = merged_df_['age']
    merged_df_['age'] = mean(age_dict.values())

    ## Sex
    sex_dict = get_dict(merged_df_['sex'])
    merged_df_['sex'] = sex_dict
    
    ## Occupation code
    occp_code_dict = get_dict(merged_df_['occp_cod'])
    merged_df_['occp_cod'] = occp_code_dict
    
    ## weight
    wt_dict = merged_df_['wt']
    merged_df_['wt'] = mean(wt_dict.values())
    
    ## reporter country 
    rp_count_dict = get_dict(merged_df_['reporter_country'])
    merged_df_['reporter_country'] = rp_count_dict

    ## occr country 
    o_count_dict = get_dict(merged_df_['occr_country'])
    merged_df_['occr_country'] = o_count_dict
    
    

# Converting to json    
    json_object = json.dumps(merged_df_, indent = 4)  
    with open("sample2.json", "w") as outfile:  
        json.dump(merged_df_, outfile)
    print(json_object) 
    
#     calculateMetrics(merged_df)
#     merged_df_["Report_Count"]= len(merged_df) 
#     print("merged_df_",merged_df_ )
#     merged_df_.to_json('testsample.json')

In [46]:
def get_dict(dicti):
    res = {}
    fin_res = {}

    k, v = list(dicti.keys()), list(dicti.values())
    print(k)
    print(v)
    
    for val in range(len(v)):
        if v[val] in res:
            res[v[val]].append(k[val])
        else:
            res[v[val]] = [k[val]]
    
    for k,v in res.items():
        fin_res[k] = len(v)
    
    
    return fin_res

In [47]:
# %debug
get_drug_reports("SALAZOPIRINA")

SALAZOPIRINA
shape:  (5, 19)
             caseid  caseversion i_f_code    event_dt      mfr_dt  \
primaryid                                                           
111223355  11122335            5        F    201504.0  20190114.0   
144785407  14478540            7        F  20161100.0  20190327.0   
159079631  15907963            1        I    201504.0  20190131.0   
159350762  15935076            2        F  20181200.0  20190207.0   
161660871  16166087            1        I  20181000.0  20190304.0   

           init_fda_dt                       mfr_num mfr_sndr   age age_cod  \
primaryid                                                                     
111223355   20150500.0  PT-ABBVIE-15K-130-1390794-00   ABBVIE  68.0      YR   
144785407   20180200.0       CA-TAKEDA-2016TUS022277   TAKEDA  61.0      YR   
159079631   20190200.0      PT-PFIZER INC-2019048131   PFIZER  68.0      YR   
159350762   20190200.0              ES-ROCHE-2261941    ROCHE  56.0      YR   
161660871   2

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


[111223355, 144785407, 159079631, 159350762, 161660871]
['F', 'F', 'I', 'F', 'I']
[111223355, 144785407, 159079631, 159350762, 161660871]
['F', 'M', 'F', 'F', 'F']
[111223355, 144785407, 159079631, 159350762, 161660871]
['CN', 'CN', 'CN', 'MD', 'CN']
[111223355, 144785407, 159079631, 159350762, 161660871]
['PT', 'CA', 'PT', 'ES', 'PT']
[111223355, 144785407, 159079631, 159350762, 161660871]
['PT', 'CA', 'PT', 'ES', 'PT']
{
    "caseid": {
        "111223355": 11122335,
        "144785407": 14478540,
        "159079631": 15907963,
        "159350762": 15935076,
        "161660871": 16166087
    },
    "caseversion": {
        "111223355": 5,
        "144785407": 7,
        "159079631": 1,
        "159350762": 2,
        "161660871": 1
    },
    "i_f_code": {
        "F": 3,
        "I": 2
    },
    "event_dt": {
        "111223355": 201504.0,
        "144785407": 20161100.0,
        "159079631": 201504.0,
        "159350762": 20181200.0,
        "161660871": 20181000.0
    },
    "mfr

In [9]:
def convertToKG(value, cod):

  # Legacy data has some weight values with a comma...
  if "," in str(value):
    value = value.replace(",", ".");

  conversionMultiplier = {
    'KG': 1,
    'LBS': 2.2,
    'GMS': 1000,
  }[cod]
  return float(value) * conversionMultiplier

def convertToYear(value, cod):
  if cod == 'DEC':
    return int(value) * 10

  conversionMultiplier = {
    'YR': 1,
    'MON': 12,
    'WK': 52,
    'DY': 365,
    'HR': 8760
  }[cod]
  return float(value) / conversionMultiplier


In [20]:
# Converting all weight attributes to Kilograms. 
def convertToKG(value, cod):

  # Legacy data has some weight values with a comma...
    if "," in str(value):
        value = value.replace(",", ".")

    conversionMultiplier = {
    'KG': 1,
    'LBS': 2.2,
    'GMS': 1000,
    }[cod]
    return float(value) * conversionMultiplier

# Converting all output results to Year
def convertToYear(value, cod):
    if cod == 'DEC':
        return int(value) * 10

    conversionMultiplier = {
    'YR': 1,
    'MON': 12,
    'WK': 52,
    'DY': 365,
    'HR': 8760
    }[cod]
    return float(value) / conversionMultiplier



In [10]:
def calculateMetrics(mergedDF):
    drugDict = {}
    drugsArray = []
    demographicsDict = {}
    totalKG = 0
    totalAge = 0
    hasWeight={"total": 0, 'M': 0, 'F': 0, "UNK": 0, 'null': 0, 'nan': 0, 'NS': 0}
    hasAge={"total": 0, 'M': 0, 'F': 0, "UNK": 0, 'null': 0, 'nan': 0, 'NS': 0}
    prescriptionReasons = {}
    adverseEffects = {}
    outcomeEffects = {}
    manufacturersDict = {}
    occupationDict = {}
    monthlyDict = {}
    countryDict = {}
    electronicSubDict = {}
    notifiedManufacturerDict = {}
    totalReports = 0
    
    
    id = 0
    print('Converting to JSON data')
   
    
    for index, row in mergedDF.iterrows():
        
        if str(row['caseid']) in demographicsDict:
              demographicsDict[str(row['caseid'])]['total'] += 1
        else:
              demographicsDict[str(row['caseid'])]={"total": 1,
          "averageWeight": 0, "averageAge": 0}
         
        
        for gender in demographicsDict:
            if isinstance(demographicsDict[gender], dict):
                
                if gender in hasWeight and hasWeight[gender] != 0:
                    avgWeight=demographicsDict[gender]["averageWeight"] / \
                    hasWeight[gender]
                    demographicsDict[gender]["averageWeight"]=avgWeight

                if gender in hasWeight and hasAge[gender] != 0:
                    avgAge=demographicsDict[gender]["averageAge"] / hasAge[gender]
                    demographicsDict[gender]["averageAge"]=avgAge

        drugsArray.append(drugDict)
        id += 1
        
        totalReports = 0
        totalKG = 0
        totalAge = 0
        hasWeight={"total": 0, 'M': 0, 'F': 0, "UNK": 0, 'null': 0, 'nan': 0, 'NS': 0}
        hasAge={"total": 0, 'M': 0, 'F': 0, "UNK": 0, 'null': 0, 'nan': 0, 'NS': 0}

        drugName = []
        drugDict = {}
#         demographicsDict = {}

    totalReports += 1
        
        
    if (row['wt'] != 'null') and (row['wt_cod'] != 'null') and not ((np.isnan(row['wt']))) and  not ((np.isnan(row['wt_cod']))) :
        kgWeight=convertToKG(row['wt'], row['wt_cod'])
        totalKG += kgWeight
        demographicsDict[str(row['caseid'])]["averageWeight"] += kgWeight
        hasWeight["total"] += 1
        hasWeight[str(row['caseid'])] += 1

    if (row['age'] != 'null') and (row['age_cod'] != 'null') and str(row['caseid']) in demographicsDict and str(row['caseid']) in hasAge:
        yrAge=convertToYear(row['age'], row['age_cod'])
        totalAge += yrAge
        demographicsDict[str(row['caseid'])]["averageAge"] += yrAge
        hasAge["total"] += 1
        hasAge[str(row['caseid'])] += 1

    if row['mfr_sndr'] in manufacturersDict:
        demographicsDict[row['mfr_sndr']] += 1
    else:
        demographicsDict[row['mfr_sndr']] = 1

    if row['to_mfr'] in notifiedManufacturerDict:
        demographicsDict[row['to_mfr']] += 1
    else:
        demographicsDict[row['to_mfr']] = 1

    if row['occp_cod'] in occupationDict:
        demographicsDict[row['occp_cod']] += 1
    else:
        demographicsDict[row['occp_cod']] = 1

    if row['reporter_country'] in countryDict:
        demographicsDict[row['reporter_country']] += 1
    else:
        demographicsDict[row['reporter_country']] = 1

    print(demographicsDict)
    
    print(drugsArray)
#     with open('drugs20' + str(year) + '.json', 'w') as outputFile:
#     json.dump(masterDict, outputFile, indent=4)

